In [1]:
import tkinter as tk
from tkinter import messagebox
from tkcalendar import Calendar
import json
import datetime
import threading
import time
from plyer import notification
from gtts import gTTS
import os

# Initialize files for storing tasks and user credentials
TASK_FILE = "tasks.json"
USER_FILE = "users.json"
tasks = []
current_user = None

def load_users():
    try:
        with open(USER_FILE, 'r') as f:
            users = json.load(f)
            if isinstance(users, list):
                for user in users:
                    if not isinstance(user, dict):
                        return []
                return users
            return []
    except (FileNotFoundError, json.JSONDecodeError):
        return []

def save_users(users):
    with open(USER_FILE, 'w') as f:
        json.dump(users, f, indent=4)

def load_tasks():
    global tasks
    try:
        with open(TASK_FILE, 'r') as f:
            all_tasks = json.load(f)
            if isinstance(all_tasks, dict):
                tasks = all_tasks.get(current_user, [])
            else:
                tasks = []
    except (FileNotFoundError, json.JSONDecodeError):
        tasks = []




def save_tasks():
    try:
        with open(TASK_FILE, 'r') as f:
            all_tasks = json.load(f)
            if not isinstance(all_tasks, dict):
                all_tasks = {}
    except FileNotFoundError:
        all_tasks = {}

    all_tasks[current_user] = tasks
    with open(TASK_FILE, 'w') as f:
        json.dump(all_tasks, f, indent=4)


def login():
    global current_user
    username = username_entry.get()
    password = password_entry.get()

    if not username or not password:
        messagebox.showwarning("Input Error", "Username and password are required.")
        return

    users = load_users()
    for user in users:
        if user['username'] == username and user['password'] == password:
            current_user = username
            messagebox.showinfo("Welcome", f"Welcome, {current_user}!")
            login_window.destroy()
            main_application()
            return

    messagebox.showerror("Login Failed", "Invalid username or password.")

def register():
    username = username_entry.get()
    password = password_entry.get()

    if not username or not password:
        messagebox.showwarning("Input Error", "Username and password are required.")
        return

    users = load_users()
    for user in users:
        if user['username'] == username:
            messagebox.showerror("Registration Failed", "Username already exists.")
            return

    users.append({'username': username, 'password': password})
    save_users(users)
    messagebox.showinfo("Registration Success", "You have successfully registered!")

def add_task():
    title = title_entry.get()
    due_date = f"{cal.get_date()} {hour_var.get()}:{minute_var.get()} {ampm_var.get()}"
    priority = priority_var.get()
    if not title:
        messagebox.showwarning("Input Error", "Task title is required.")
        return

    task = {
        "title": title,
        "due_date": due_date,
        "priority": priority,
        "status": "pending",
        "timestamp": datetime.datetime.now().isoformat()
    }
    tasks.append(task)
    save_tasks()
    update_task_list()
    title_entry.delete(0, tk.END)

def update_task():
    selected_task = get_selected_task(pending_listbox)
    if not selected_task:
        selected_task = get_selected_task(overdue_listbox)
    if not selected_task:
        messagebox.showwarning("Selection Error", "Select a task to update.")
        return

    index = selected_task[0]
    tasks[index]["title"] = title_entry.get()
    tasks[index]["due_date"] = f"{cal.get_date()} {hour_var.get()}:{minute_var.get()} {ampm_var.get()}"
    tasks[index]["priority"] = priority_var.get()
    save_tasks()
    update_task_list()

def delete_task():
    selected_task = get_selected_task(pending_listbox)
    if not selected_task:
        selected_task = get_selected_task(overdue_listbox)
    if not selected_task:
        messagebox.showwarning("Selection Error", "Select a task to delete.")
        return

    index = selected_task[0]
    tasks.pop(index)
    save_tasks()
    update_task_list()

def mark_as_completed():
    selected_task = get_selected_task(pending_listbox)
    if not selected_task:
        selected_task = get_selected_task(overdue_listbox)
    if not selected_task:
        messagebox.showwarning("Selection Error", "Select a task to mark as completed.")
        return

    index = selected_task[0]
    tasks[index]["status"] = "completed"
    save_tasks()
    update_task_list()

def get_selected_task(listbox):
    selected_task = listbox.curselection()
    if selected_task:
        return selected_task
    return None

def update_task_list():
    pending_tasks = []
    overdue_tasks = []
    completed_tasks = []
    current_time = datetime.datetime.now()

    for task in tasks:
        task_due = datetime.datetime.strptime(task["due_date"], "%m/%d/%y %I:%M %p")
        if task["status"] == "completed":
            completed_tasks.append(task)
        elif task_due < current_time:
            overdue_tasks.append(task)
        else:
            pending_tasks.append(task)

    update_listbox(pending_tasks, pending_listbox)
    update_listbox(overdue_tasks, overdue_listbox)
    update_listbox(completed_tasks, completed_listbox)

def update_listbox(task_list, listbox):
    listbox.delete(0, tk.END)
    for task in task_list:
        task_info = f"{task['title']} | Due: {task['due_date']} | Priority: {task['priority']} | Status: {task['status']}"
        listbox.insert(tk.END, task_info)

def reminder():
    global tasks
    while True:
        current_time = datetime.datetime.now()
        for task in tasks:
            if task["status"] == "pending":
                task_due = datetime.datetime.strptime(task["due_date"], "%m/%d/%y %I:%M %p")
                time_difference = (task_due - current_time).total_seconds()

                if 0 < time_difference <= 600:
                    notification.notify(
                        title='Task Reminder',
                        message=f"Task '{task['title']}' is due in less than 10 minutes!",
                        timeout=10
                    )
                    play_sound(task['title'])
        
        time.sleep(60)

def play_sound(task_title):
    message = f"Reminder for your task: {task_title}"
    tts = gTTS(text=message, lang='en', slow=False)
    tts.save("reminder.mp3")
    os.system("start reminder.mp3")

def set_reminder():
    try:
        reminder_time_minutes = int(entry.get())
        if reminder_time_minutes <= 0:
            messagebox.showerror("Invalid Time", "Please enter a positive number.")
            return
        
        reminder_time_seconds = reminder_time_minutes * 60

        messagebox.showinfo("Reminder Set", f"Reminder set for {reminder_time_minutes} minutes from now.")
        threading.Thread(target=reminder_alarm, args=(reminder_time_seconds,), daemon=True).start()
    
    except ValueError:
        messagebox.showerror("Invalid Input", "Please enter a valid number.")

def reminder_alarm(reminder_time):
    time.sleep(reminder_time)
    notification.notify(
        title='General Reminder',
        message='Time to check your reminder!',
        timeout=10
    )
    play_sound("General Reminder")
def login_window():
    global username_entry, password_entry, login_window
    login_window = tk.Tk()
    login_window.title("Login")

    # Set the window size and position it in the center of the screen
    window_width = 300
    window_height = 200
    screen_width = login_window.winfo_screenwidth()
    screen_height = login_window.winfo_screenheight()
    position_top = int(screen_height / 2 - window_height / 2)
    position_right = int(screen_width / 2 - window_width / 2)
    login_window.geometry(f'{window_width}x{window_height}+{position_right}+{position_top}')

    # Define the font
    font = ("Helvetica", 14)

    # Add padding around all widgets and center them using sticky
    login_window.grid_rowconfigure(0, weight=1)
    login_window.grid_rowconfigure(4, weight=1)
    login_window.grid_columnconfigure(0, weight=1)
    login_window.grid_columnconfigure(2, weight=1)

    tk.Label(login_window, text="Username", font=font).grid(row=1, column=1, padx=10, pady=10, sticky='e')
    username_entry = tk.Entry(login_window, font=font)
    username_entry.grid(row=1, column=2, padx=10, pady=10, sticky='w')

    tk.Label(login_window, text="Password", font=font).grid(row=2, column=1, padx=10, pady=10, sticky='e')
    password_entry = tk.Entry(login_window, show="*", font=font)
    password_entry.grid(row=2, column=2, padx=10, pady=10, sticky='w')

    tk.Button(login_window, text="Login", command=login, font=font).grid(row=3, column=1, padx=10, pady=10, sticky='e')
    tk.Button(login_window, text="Register", command=register, font=font).grid(row=3, column=2, padx=10, pady=10, sticky='w')

    login_window.mainloop()

def main_application():
    global title_entry, cal, hour_var, minute_var, ampm_var, priority_var
    global pending_listbox, overdue_listbox, completed_listbox, entry

    root = tk.Tk()
    root.title("Task Manager & Reminder App")
    
    tk.Label(root, text="Task Title").grid(row=0, column=0, pady=10)
    title_entry = tk.Entry(root, width=50)
    title_entry.grid(row=0, column=1, padx=10, pady=10)

    tk.Label(root, text="Due Date").grid(row=1, column=0)
    mindate = datetime.date.today()
    cal = Calendar(root, selectmode="day", mindate=mindate)
    cal.grid(row=1, column=1, padx=10, pady=10)

    tk.Label(root, text="Due Time (HH:MM)").grid(row=2, column=0)
    hour_var = tk.StringVar(value="12")
    minute_var = tk.StringVar(value="00")
    ampm_var = tk.StringVar(value="AM")

    hour_spinbox = tk.Spinbox(root, from_=1, to=12, textvariable=hour_var, width=3)
    minute_spinbox = tk.Spinbox(root, from_=0, to=59, textvariable=minute_var, width=3)

    hour_spinbox.grid(row=2, column=1, padx=(10, 5), pady=10, sticky='w')
    minute_spinbox.grid(row=2, column=1, padx=(5, 10), pady=10, sticky='e')

    ampm_menu = tk.OptionMenu(root, ampm_var, "AM", "PM")
    ampm_menu.grid(row=2, column=1, padx=(50, 0))

    tk.Label(root, text="Priority").grid(row=3, column=0)
    priority_var = tk.IntVar(value=1)
    priority_spinbox = tk.Spinbox(root, from_=1, to=10, textvariable=priority_var)
    priority_spinbox.grid(row=3, column=1)

    add_button = tk.Button(root, text="Add Task", command=add_task)
    add_button.grid(row=4, column=0)

    update_button = tk.Button(root, text="Update Task", command=update_task)
    update_button.grid(row=4, column=1)

    delete_button = tk.Button(root, text="Delete Task", command=delete_task)
    delete_button.grid(row=5, column=0)

    complete_button = tk.Button(root, text="Mark as Completed", command=mark_as_completed)
    complete_button.grid(row=5, column=1)

    tk.Label(root, text="Pending Tasks").grid(row=6, column=0)
    pending_listbox = tk.Listbox(root, width=60, height=10)
    pending_listbox.grid(row=7, column=0, columnspan=2, padx=10, pady=10)

    tk.Label(root, text="Overdue Tasks").grid(row=6, column=2)
    overdue_listbox = tk.Listbox(root, width=60, height=10)
    overdue_listbox.grid(row=7, column=2, columnspan=2, padx=10, pady=10)

    tk.Label(root, text="Completed Tasks").grid(row=8, column=0)
    completed_listbox = tk.Listbox(root, width=60, height=10)
    completed_listbox.grid(row=9, column=0, columnspan=2, padx=10, pady=10)

    tk.Label(root, text="Set General Reminder Time (Minutes)").grid(row=10, column=0)
    entry = tk.Entry(root, width=20)
    entry.grid(row=10, column=1, padx=10, pady=10)

    reminder_button = tk.Button(root, text="Set Reminder", command=set_reminder)
    reminder_button.grid(row=10, column=2)

    global tasks
    load_tasks()  # Ensure tasks are loaded
    update_task_list()

    threading.Thread(target=reminder, daemon=True).start()

    root.mainloop()
login_window()
